In [1]:
import pandas as pd

# Lebanon arms

## Load data from DVC Google sheet

In [2]:
df = (
    pd.read_csv(
        "data/raw/DVC_DataCollection_FrenchArms_LB01_2020 - Content.csv",
        usecols=[
            1,
            4,
            7,
            10,
            13,
            14,
            *range(17, 20),
            *range(22, 25),
            *range(27, 30),
            *range(32, 35),
            *range(37, 40),
            *range(42, 45),
            *range(47, 50),
        ],
        header=0,
        names=[
            "content_code",
            "publication_date",
            "verification_status",
            "event_date",
            "latitude",
            "longitude",
            "equipment1_type",
            "equipment1_manufacturer",
            "equipment1_model",
            "equipment2_type",
            "equipment2_manufacturer",
            "equipment2_model",
            "equipment3_type",
            "equipment3_manufacturer",
            "equipment3_model",
            "equipment4_type",
            "equipment4_manufacturer",
            "equipment4_model",
            "equipment5_type",
            "equipment5_manufacturer",
            "equipment5_model",
            "equipment6_type",
            "equipment6_manufacturer",
            "equipment6_model",
            "equipment7_type",
            "equipment7_manufacturer",
            "equipment7_model",
        ],
        parse_dates=[1],
    )
    .astype({"latitude": "float", "longitude": "float"})
    .assign(event_date=lambda df: pd.to_datetime(df["event_date"], errors="coerce"))
    .query("verification_status == 'Verified' ")
    .query("content_code != 'LB068' & content_code != 'LB058'")
    .reset_index(drop=True)
)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   content_code             107 non-null    object        
 1   publication_date         107 non-null    datetime64[ns]
 2   verification_status      107 non-null    object        
 3   event_date               98 non-null     datetime64[ns]
 4   latitude                 95 non-null     float64       
 5   longitude                95 non-null     float64       
 6   equipment1_type          90 non-null     object        
 7   equipment1_manufacturer  90 non-null     object        
 8   equipment1_model         90 non-null     object        
 9   equipment2_type          38 non-null     object        
 10  equipment2_manufacturer  38 non-null     object        
 11  equipment2_model         38 non-null     object        
 12  equipment3_type          24 non-null

,content_code,publication_date,verification_status,event_date,latitude,longitude,equipment1_type,equipment1_manufacturer,equipment1_model,equipment2_type,...,equipment4_model,equipment5_type,equipment5_manufacturer,equipment5_model,equipment6_type,equipment6_manufacturer,equipment6_model,equipment7_type,equipment7_manufacturer,equipment7_model
0,LB001,2020-01-18,Verified,2020-01-18,33.897517,35.507076,Vehicle-mounted weapon,SAE Alsetex,Land Cougar 12,Vehicle,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LB002,2020-08-08,Verified,2020-08-08,33.897800,35.506168,Vehicle,IVECO,VM 90,Vehicle-mounted weapon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LB003,2020-08-09,Verified,2020-08-09,33.898331,35.503759,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LB004,2020-01-09,Verified,2020-01-09,33.897517,35.507076,Vehicle,IVECO,VBTP-MR Guarani 6x6,Vehicle-mounted weapon,...,CM4/CM6,Vehicle,Other,Water Cannon,NaN,NaN,NaN,NaN,NaN,NaN
4,LB005_1,2020-01-16,Verified,2020-01-15,33.879776,35.493568,Launcher,SAE Alsetex,Cougar 56mm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Transform and write to CSV for Flourish treemap

In [3]:
df_types = (
    df.copy()
    .melt(
        id_vars=["content_code", "event_date"],
        value_vars=[
            "equipment1_type",
            "equipment2_type",
            "equipment3_type",
            "equipment4_type",
            "equipment5_type",
            "equipment6_type",
            "equipment7_type",
        ],
        var_name="equipment_no",
        value_name="type",
    )
    .query("type == type")
    .sort_values(["content_code", "event_date", "equipment_no"])
    .reset_index(drop=True)
)

df_types.info()
df_types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   content_code  175 non-null    object        
 1   event_date    162 non-null    datetime64[ns]
 2   equipment_no  175 non-null    object        
 3   type          175 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.6+ KB


,content_code,event_date,equipment_no,type
0,LB001,2020-01-18,equipment1_type,Vehicle-mounted weapon
1,LB001,2020-01-18,equipment2_type,Vehicle
2,LB002,2020-08-08,equipment1_type,Vehicle
3,LB002,2020-08-08,equipment2_type,Vehicle-mounted weapon
4,LB004,2020-01-09,equipment1_type,Vehicle
...,...,...,...,...
170,LB107,2019-12-14,equipment2_type,Munition
171,LB107,2019-12-14,equipment3_type,Munition
172,LB124,2019-12-14,equipment1_type,Vehicle
173,LB124,2019-12-14,equipment2_type,Vehicle


In [4]:
df_manufacturers = (
    df.copy()
    .melt(
        id_vars=["content_code", "event_date"],
        value_vars=[
            "equipment1_manufacturer",
            "equipment2_manufacturer",
            "equipment3_manufacturer",
            "equipment4_manufacturer",
            "equipment5_manufacturer",
            "equipment6_manufacturer",
            "equipment7_manufacturer",
        ],
        var_name="manufacturer_no",
        value_name="manufacturer",
    )
    .query("manufacturer == manufacturer")
    .sort_values(["content_code", "event_date", "manufacturer_no"])
    .reset_index(drop=True)
)

df_manufacturers.info()
df_manufacturers

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   content_code     175 non-null    object        
 1   event_date       162 non-null    datetime64[ns]
 2   manufacturer_no  175 non-null    object        
 3   manufacturer     175 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.6+ KB


,content_code,event_date,manufacturer_no,manufacturer
0,LB001,2020-01-18,equipment1_manufacturer,SAE Alsetex
1,LB001,2020-01-18,equipment2_manufacturer,IVECO
2,LB002,2020-08-08,equipment1_manufacturer,IVECO
3,LB002,2020-08-08,equipment2_manufacturer,SAE Alsetex
4,LB004,2020-01-09,equipment1_manufacturer,IVECO
...,...,...,...,...
170,LB107,2019-12-14,equipment2_manufacturer,SAE Alsetex
171,LB107,2019-12-14,equipment3_manufacturer,SAE Alsetex
172,LB124,2019-12-14,equipment1_manufacturer,IVECO
173,LB124,2019-12-14,equipment2_manufacturer,IVECO


In [5]:
df_joined = pd.concat(
    [
        df_types[["content_code", "event_date", "type"]],
        df_manufacturers[["manufacturer"]],
    ],
    axis=1,
).reset_index(drop=True)

df_joined.info()
df_joined

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   content_code  175 non-null    object        
 1   event_date    162 non-null    datetime64[ns]
 2   type          175 non-null    object        
 3   manufacturer  175 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 5.6+ KB


,content_code,event_date,type,manufacturer
0,LB001,2020-01-18,Vehicle-mounted weapon,SAE Alsetex
1,LB001,2020-01-18,Vehicle,IVECO
2,LB002,2020-08-08,Vehicle,IVECO
3,LB002,2020-08-08,Vehicle-mounted weapon,SAE Alsetex
4,LB004,2020-01-09,Vehicle,IVECO
...,...,...,...,...
170,LB107,2019-12-14,Munition,SAE Alsetex
171,LB107,2019-12-14,Munition,SAE Alsetex
172,LB124,2019-12-14,Vehicle,IVECO
173,LB124,2019-12-14,Vehicle,IVECO


In [6]:
df_joined[["content_code", "type", "manufacturer"]].to_csv(
    "data/processed/lebanon-arms-treemap-en.csv", index=None
)

## Transform and write to CSV for map

### Define function to concatenate text for map tooltips

In [7]:
def tooltip(items_list):
    text_list = []
    
    for item in items_list:
        item_count = items_list.count(item)
        item_text = f"{item_count} x {item}"
        
        if item_count > 1 and item_text not in text_list:
            text_list.append(item_text)
        elif item_count == 1:
            text_list.append(item)
        
    return ", ".join(text_list).capitalize()

In [8]:
df_map = (
    df.copy()
    .assign(
        types_list_nans=lambda df: df[
            [
                "equipment1_type",
                "equipment2_type",
                "equipment3_type",
                "equipment4_type",
                "equipment5_type",
                "equipment6_type",
                "equipment7_type",
            ]
        ].values.tolist(),
        types_list=lambda df: df["types_list_nans"].apply(
            lambda x: sorted([d for d in x if str(d) != "nan"])
        ),
        tooltip=lambda df: df["types_list"].apply(lambda x: tooltip(x)),
        types_count=lambda df: df["types_list"].apply(lambda x: len(x)),
        color="#000000",
        name=" ",
    )
    .query("latitude == latitude & longitude == longitude & types_count > 0")
    .reset_index(drop=True)
)

df_map.info()
df_map.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   content_code             78 non-null     object        
 1   publication_date         78 non-null     datetime64[ns]
 2   verification_status      78 non-null     object        
 3   event_date               70 non-null     datetime64[ns]
 4   latitude                 78 non-null     float64       
 5   longitude                78 non-null     float64       
 6   equipment1_type          78 non-null     object        
 7   equipment1_manufacturer  78 non-null     object        
 8   equipment1_model         78 non-null     object        
 9   equipment2_type          35 non-null     object        
 10  equipment2_manufacturer  35 non-null     object        
 11  equipment2_model         35 non-null     object        
 12  equipment3_type          21 non-null  

,content_code,publication_date,verification_status,event_date,latitude,longitude,equipment1_type,equipment1_manufacturer,equipment1_model,equipment2_type,...,equipment6_model,equipment7_type,equipment7_manufacturer,equipment7_model,types_list_nans,types_list,tooltip,types_count,color,name
0,LB001,2020-01-18,Verified,2020-01-18,33.897517,35.507076,Vehicle-mounted weapon,SAE Alsetex,Land Cougar 12,Vehicle,...,NaN,NaN,NaN,NaN,"[Vehicle-mounted weapon, Vehicle, nan, nan, na...","[Vehicle, Vehicle-mounted weapon]","Vehicle, vehicle-mounted weapon",2,#000000,
1,LB002,2020-08-08,Verified,2020-08-08,33.897800,35.506168,Vehicle,IVECO,VM 90,Vehicle-mounted weapon,...,NaN,NaN,NaN,NaN,"[Vehicle, Vehicle-mounted weapon, nan, nan, na...","[Vehicle, Vehicle-mounted weapon]","Vehicle, vehicle-mounted weapon",2,#000000,
2,LB004,2020-01-09,Verified,2020-01-09,33.897517,35.507076,Vehicle,IVECO,VBTP-MR Guarani 6x6,Vehicle-mounted weapon,...,NaN,NaN,NaN,NaN,"[Vehicle, Vehicle-mounted weapon, Launcher, Mu...","[Launcher, Munition, Vehicle, Vehicle, Vehicle...","Launcher, munition, 2 x vehicle, vehicle-mount...",5,#000000,
3,LB005_1,2020-01-16,Verified,2020-01-15,33.879776,35.493568,Launcher,SAE Alsetex,Cougar 56mm,NaN,...,NaN,NaN,NaN,NaN,"[Launcher, nan, nan, nan, nan, nan, nan]",[Launcher],Launcher,1,#000000,
4,LB005_2,2020-01-16,Verified,2020-01-15,33.879205,35.495533,Launcher,SAE Alsetex,Chouka 56mm,NaN,...,NaN,NaN,NaN,NaN,"[Launcher, nan, nan, nan, nan, nan, nan]",[Launcher],Launcher,1,#000000,


In [9]:
df_map[
    ["content_code", "event_date", "longitude", "latitude", "color", "name", "tooltip"]
].to_csv("data/processed/lebanon-arms-map.csv", index=False)

## Filter on event date for interview dates

In [10]:
dates = [
    pd.to_datetime("2019-12-14"),
    pd.to_datetime("2020-01-18"),
    pd.to_datetime("2020-01-19"),
    pd.to_datetime("2020-08-08"),
]

dates

[Timestamp('2019-12-14 00:00:00'),
 Timestamp('2020-01-18 00:00:00'),
 Timestamp('2020-01-19 00:00:00'),
 Timestamp('2020-08-08 00:00:00')]

In [11]:
df_interviews = (
    df_joined.copy()
    .query("event_date in @dates")
    .sort_values(["event_date", "content_code"])
#     .query("event_date == '2020-08-08'")
    .reset_index(drop=True)
)

df_interviews.info()
df_interviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   content_code  108 non-null    object        
 1   event_date    108 non-null    datetime64[ns]
 2   type          108 non-null    object        
 3   manufacturer  108 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.5+ KB


,content_code,event_date,type,manufacturer
0,LB013_1,2019-12-14,Launcher,SAE Alsetex
1,LB013_2,2019-12-14,Launcher,SAE Alsetex
2,LB013_3,2019-12-14,Vehicle-mounted weapon,SAE Alsetex
3,LB013_3,2019-12-14,Vehicle,IVECO
4,LB017,2019-12-14,Launcher,SAE Alsetex


In [12]:
df_interviews.groupby(["type"])["manufacturer"].count()

type
Launcher                  24
Munition                  16
Vehicle                   40
Vehicle-mounted weapon    28
Name: manufacturer, dtype: int64

In [13]:
df_joined.groupby("type")["manufacturer"].count()

type
Launcher                  48
Munition                  23
Vehicle                   67
Vehicle-mounted weapon    37
Name: manufacturer, dtype: int64

In [14]:
df.query("event_date == '2020-01-19'")

,content_code,publication_date,verification_status,event_date,latitude,longitude,equipment1_type,equipment1_manufacturer,equipment1_model,equipment2_type,...,equipment4_model,equipment5_type,equipment5_manufacturer,equipment5_model,equipment6_type,equipment6_manufacturer,equipment6_model,equipment7_type,equipment7_manufacturer,equipment7_model
54,LB063,2020-01-19,Verified,2020-01-19,33.898139,35.504827,Munition,Unknown,Unknown,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,LB104,2020-01-19,Verified,2020-01-19,33.898075,35.504838,Munition,SAPL,Gomm-Cogne Balle Cal. 12/67,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,LB105,2020-01-19,Verified,2020-01-19,33.898019,35.505330,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,LB121,2020-01-19,Verified,2020-01-19,33.898129,35.504772,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_interviews.to_csv("data/processed/lebanon-arms-interview-dates.csv", index=False)